# Creeps. Reading a .bib file, matching by name, and returning a message.


TODO

- if there are no bib entries found, pass. make that clear. something like: 
    - if no bib file, send as a warning: 'no .bib file found'. 
    - if empty bib file found, pass. 
    - if no matches found, pass

- the csv module will likely run on overleaf just fine: https://docs.python.org/3/library/csv.html

- create an environment for this project (BUT, we will try to call as few packages as possible).

- if someone's name is a pain in the arse, (i.e. not made with the characters a-z), what the heck do i do?
  author={Blais, Andr{\'e}},
  author={Cancela, Jo{\~a}o and Geys, Benny},
  author={Frank, Richard W and Mart{\'\i}nez i Coma, Ferran},

- find bib files that aren't generated by google scholar

- some author entries have double curly braces, i'm sure i'm stuffing them up: author={{World Food Summit}},


- just take names as given, don't edit. match. later on, can think about adding different options using TheFuzz and leivenstein distance stuff as well as dropping middle names and run:
    - if it's got a "." kill the "."
    - if it's got a word that is only 1 leter long, kill the word (that's gonna be an initial).
    - kill " Jr "; " Sr " .... any others like that?

In [2]:
import pandas as pd

# Take the database of offenders and return names. Turn this into a csv, just rip out what i want, so that later, i can do this without pandas.

df = pd.read_csv('asmd_incidents.csv', usecols=['Person', 'Institution', 'Original Link(s)']) # Just load the usable columns.
df = df[(df.Person != 'NAME UNKNOWN') & ~df.Person.isna()]

df['Person'] = df['Person'].astype(str).str.lower()
# df['Person'] = df['Person'].str.replace(',',"") # This is necessary for later on. There's one bloke who is so and so Junior, and has a comma in his name.
offenders = [offender for offender in list(df.Person)] # Offenders isn't used, so remove.

df.columns = ['person', 'institution', 'source']
df.to_csv('asmd.csv', index = False, sep = "\t") # Some of the names have a comma, and even some of URLs, so this helps for the string splits.

In [6]:
# Code before I turn it into a function.

# Take a .bib file, return the authors.
bibfile = "examples"
if not bibfile.endswith(".bib"):
    bibfile = bibfile + ".bib"

class keyauthor:
    def __init__(self, keys = [], authors = []):
        self.keys = keys
        self.authors = authors

    def update(self, line):
        if line.strip().startswith("@"):
            self.keys.append(line.strip().split("{")[1].rstrip(',').strip().lower() )
        elif line.strip().split("=")[0].lower().__contains__("author"):
            self.authors.append(line.strip().split("{")[1].split("}")[0].split(" and "))


ka = keyauthor()
with open(bibfile) as refs:
    for line in refs:
        ka.update(line)

if len(ka.authors) == 0:
    pass

# If the name is of the form: "last_name, first_name (middle_names)", flip it around and remove the comma, so it is of the form: "first_name (middle names) last_name"
for i in range(len(ka.authors)):
    ka.authors[i] = [x.split(",")[1].strip().lower() + " " + x.split(",")[0].strip().lower() 
                  if x.__contains__(",") 
                  else x 
                  for x in ka.authors[i]
                  ]

names = [item.lower() for sublist in ka.authors for item in sublist]

# Turn to dictionary
bibd = dict(zip(ka.keys, ka.authors))

# Take the csv, using vanilla python, return a dictionary with names and links.
entries = []

# Open the CSV file in read mode
with open('asmd.csv', 'r') as file:
    for line in file.readlines():
        entries.append([x.strip() for x in line.split('\t')])

# Create a dictionary of the above, but first creating separate lists.
offenders, institutions, sources = zip(*entries)
offd = dict(zip(offenders, zip(institutions, sources)))

# This works nicely only for exact name matching.
warnings = set([name for name in names if name in offenders])

# Now, match the database and the .bib file.
# Return the relevant bibliography identifiers for each offender.
article_ids, institutions, sources = [], [], [] # this overwrites previous stuff. ugly, i know.
for w in warnings:
    art_id = []
    for k, v in bibd.items():
        if w in v:
            art_id.append(k)

    article_ids.append(art_id)

    for k, v in offd.items():
        if k == w:
            institutions.append(offd.get(w)[0])
            sources.append(offd.get(w)[1])

for i in range(len(warnings)):
    message = f"-- {list(warnings)[i]} [{institutions[i]}; {article_ids[i]}] is matched to the database of offenders. For details, see " 
    print(message)

    # Now, cycle through, print it if it's the final source in the list with a full stop after, otherwise print with a comma.
    for k, j in enumerate(sources[i].split(";")):
        if k != len(sources[i].split(";")) -1:
            details = f"\href{{{j}}}{{here}}, "
            print(details)
        else:
            details = f"\href{{{j}}}{{here}}."
            print(details)

    print('\n')    

In [4]:
# Turning it into a function

def creeps(bibfile):

    # Take a .bib file, return the authors.
    if not bibfile.endswith(".bib"):
        bibfile = bibfile + ".bib"

    class keyauthor:
        def __init__(self, keys = [], authors = []):
            self.keys = keys
            self.authors = authors

        def update(self, line):
            if line.strip().startswith("@"):
                self.keys.append(line.strip().split("{")[1].rstrip(',').strip().lower() )
            elif line.strip().split("=")[0].lower().__contains__("author"):
                self.authors.append(line.strip().split("{")[1].split("}")[0].split(" and "))


    ka = keyauthor()
    try:
        with open(bibfile) as refs:
            for line in refs:
                ka.update(line)
    except FileNotFoundError:
        print("TODO -- throw a suitable warning for inputting an invalid/unfindable .bib file")
        return

    if len(ka.authors) == 0:
        print("TODO -- throw a warning for having an empty .bib file")
        return # This ensures the rest of the code isn't run if the .bib file is empty

    # If the name is of the form: "last_name, first_name (middle_names)", flip it around and remove the comma, so it is of the form: "first_name (middle names) last_name"
    for i in range(len(ka.authors)):
        ka.authors[i] = [x.split(",")[1].strip().lower() + " " + x.split(",")[0].strip().lower() 
                    if x.__contains__(",") 
                    else x 
                    for x in ka.authors[i]
                    ]

    names = [item.lower() for sublist in ka.authors for item in sublist]

    # Turn to dictionary
    bibd = dict(zip(ka.keys, ka.authors))

    # Take the csv, using vanilla python, return a dictionary with names and links.
    entries = []

    # Open the CSV file in read mode
    with open('asmd.csv', 'r') as file:
        for line in file.readlines():
            entries.append([x.strip() for x in line.split('\t')])

    # Create a dictionary of the above, but first creating separate lists.
    offenders, institutions, sources = zip(*entries)
    offd = dict(zip(offenders, zip(institutions, sources)))

    # This works nicely only for exact name matching.
    warnings = set([name for name in names if name in offenders])

    # Now, match the database and the .bib file.
    # Return the relevant bibliography identifiers for each offender.
    article_ids, institutions, sources = [], [], [] # this overwrites previous stuff. ugly, i know.
    for w in warnings:
        art_id = []
        for k, v in bibd.items():
            if w in v:
                art_id.append(k)

        article_ids.append(art_id)

        for k, v in offd.items():
            if k == w:
                institutions.append(offd.get(w)[0])
                sources.append(offd.get(w)[1])

    for i in range(len(warnings)):
        message = f"-- {list(warnings)[i]} [{institutions[i]}; {article_ids[i]}] is matched to the database of offenders. For details, see " 
        print(message)

        # Now, cycle through, print it if it's the final source in the list with a full stop after, otherwise print with a comma.
        for k, j in enumerate(sources[i].split(";")):
            if k != len(sources[i].split(";")) -1:
                details = f"\href{{{j}}}{{here}}, "
                print(details)
            else:
                details = f"\href{{{j}}}{{here}}."
                print(details)

        print('\n')    


In [8]:
creeps('examples copy')

-- reidar dittmann [St. Olaf College; ['dittmann1976strindberg']] is matched to the database of offenders. For details, see 
\href{https://www.insidehighered.com/news/2017/03/15/how-st-olaf-scrubbed-building-name-revered-professor-accused-sexual-misconduct}{here}.


-- nadeem naqvi [University of Georgia; ['naqvi1995power', 'batra1987urban']] is matched to the database of offenders. For details, see 
\href{https://www.usg.edu/assets/regents/documents/board_meetings/aug99min.pdf}{here}.




In [17]:
# Reading from an online file, without using external packages. 

from urllib.request import urlopen
with urlopen("https://www.example.com") as response:
     body = response.read()

print(body[:15])


b'<!doctype html>'


In [16]:
import requests
url = 'https://www.geeksforgeeks.org'
response = requests.get(url).text
print(response)


<!DOCTYPE html>
<!--[if IE 7]>
<html class="ie ie7" lang="en-US" prefix="og: http://ogp.me/ns#">
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8" lang="en-US" prefix="og: http://ogp.me/ns#">
<![endif]-->
<!--[if !(IE 7) | !(IE 8)  ]><!-->
<html lang="en-US" prefix="og: http://ogp.me/ns#" >

<!--<![endif]-->
<head>
<meta charset="UTF-8" />
<meta name="keywords" content="Data Structures, Algorithms, Python, Java, C, C++, JavaScript, Android Development, SQL, Data Science, Machine Learning, PHP, Web Development, System Design, Tutorial, Technical Blogs, Interview Experience, Interview Preparation, Programming, Competitive Programming, SDE Sheet, Job-a-thon, Coding Contests, GATE CSE, HTML, CSS, React, NodeJS, Placement, Aptitude, Quiz, Computer Science, Programming Examples, GeeksforGeeks Courses, Puzzles">
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=0.5, maximum-scale=3.0"> 
<meta property="og:description" name="description" content="A Computer Sc